# Import Packages

In [ ]:
# Import packages
!pip install scikit-multilearn
import pandas as pd
from scipy.sparse import hstack
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import os
import re
from collections import Counter
import random
import pickle
import string
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import NuSVC, LinearSVC, SVC
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from skmultilearn.model_selection import iterative_train_test_split

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


     |████████████████████████████████| 92kB 2.5MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
dir = "drive/My Drive/BT4221/Data/"

Mounted at /content/drive


## Data Preprocessing


### Load preprocessed training dataframe

In [ ]:
df = pd.read_csv(dir+"cleaned_train.csv")
df = df[~df['processed_text'].isnull()]
df.reset_index(drop=True,inplace=True)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,word_count,char_count,word_density,total_length,capitals,capitals_prop,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,num_unique_words,prop_unique_words,processed_text,polarity,subjectivity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,43,223,0.191964,264,17,0.064394,0,1,6,0,41,0.953488,explanation edits made username hardcore metal...,0.136364,0.454545
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,17,95,0.177083,112,8,0.071429,1,0,5,0,17,1.000000,daww match background colour seemingly stuck t...,0.200000,0.200000
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,42,192,0.217617,233,4,0.017167,0,0,4,0,39,0.928571,hey man really trying edit war guy constantly ...,0.150000,0.383333
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,113,511,0.220703,622,11,0.017685,0,0,6,0,82,0.725664,ca nt make real suggestion improvement wondere...,0.183333,0.297222
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,13,55,0.232143,67,2,0.029851,0,1,3,0,13,1.000000,sir hero chance remember page,0.000000,0.000000


In [ ]:
#pickle.dump(count_vect,open('/content/drive/My Drive/BT4221/Models/count_vect_v1.sav','wb'))
#pickle.dump(tf,open('/content/drive/My Drive/BT4221/Models/tf_v1.sav','wb'))

### Load Count_vect & tfidf to calculate TFIDF Matrix

In [ ]:
count_vect = pickle.load(open('/content/drive/My Drive/BT4221/Models/count_vect_v1.sav','rb'))
tf = pickle.load(open('/content/drive/My Drive/BT4221/Models/tf_v1.sav','rb'))
X= df['processed_text']
#count_vect = CountVectorizer()
X_count = count_vect.transform(X)
#tf = TfidfTransformer()
X = tf.fit_transform(X_count)

In [ ]:
print(X.shape)
print(df.shape)

(159502, 188826)
(159502, 23)


### Append Added_features using HStack

In [ ]:
added_features = pd.DataFrame(df.iloc[:, 8:23])
added_features.drop(columns='processed_text',inplace=True)
for column in added_features.columns:
    print(column)
    var_sparse = StandardScaler().fit_transform(added_features[column].values[:, None])
    #df[column] = var_sparse
    X = hstack((X, var_sparse.astype(float)),format='csr')
print(X.shape)
df.head()

word_count
char_count
word_density
total_length
capitals
capitals_prop
num_exclamation_marks
num_question_marks
num_punctuation
num_symbols
num_unique_words
prop_unique_words
polarity
subjectivity
(159502, 188840)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,word_count,char_count,word_density,total_length,capitals,capitals_prop,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,num_unique_words,prop_unique_words,processed_text,polarity,subjectivity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,43,223,0.191964,264,17,0.064394,0,1,6,0,41,0.953488,explanation edits made username hardcore metal...,0.136364,0.454545
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,17,95,0.177083,112,8,0.071429,1,0,5,0,17,1.000000,daww match background colour seemingly stuck t...,0.200000,0.200000
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,42,192,0.217617,233,4,0.017167,0,0,4,0,39,0.928571,hey man really trying edit war guy constantly ...,0.150000,0.383333
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,113,511,0.220703,622,11,0.017685,0,0,6,0,82,0.725664,ca nt make real suggestion improvement wondere...,0.183333,0.297222
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,13,55,0.232143,67,2,0.029851,0,1,3,0,13,1.000000,sir hero chance remember page,0.000000,0.000000


In [ ]:
#df.to_csv(dir+"cleaned_train_v1.csv")

## Model Training

### Train-Test ShuffleSplit

In [ ]:
y = pd.DataFrame(df.iloc[:, 2:8]) #obtain labels dataframe
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.model_selection import ShuffleSplit

skf = ShuffleSplit(n_splits=1,test_size=0.2, random_state=42)
for train_id, test_id in skf.split(X,y):
    X_train, X_val = X[train_id], X[test_id]
    y_train, y_val = y.loc[train_id,:], y.loc[test_id,:]
    X_val_original = df['comment_text'][test_id]
X_val_original

63002                 fuck you bizitch \n\nYo fuck u fuck u
5813      "HUNK should have his own page separate from m...
26667     "\n Why do you classify the Liberal Democrats ...
148285    , 15 September 2013 (UTC)\nI have already agre...
57628     "\nGave a couple stories a listen and the repo...
                                ...                        
80579     "\n\nThat link you are looking for is now in t...
65660     "\n\nPersonally, I am shocked that this sort o...
138620    Any notional improvement was purely accidental...
36796     Good evening, Terrence! \n\nIn the UK, the tim...
139959    Oh not you again!!! And so? After the block ha...
Name: comment_text, Length: 31901, dtype: object

### Iterative Splits

In [ ]:
y = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

### Multi-label Multiclass *Learning*

In [ ]:
#!pip install scikit-multilearn

model = SVC(probability=True, C=0.5, kernel='rbf', decision_function_shape= 'ovr',degree=3, gamma=0.5,shrinking= True)
#model = XGBClassifier()

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train, y_train)

BinaryRelevance(classifier=XGBClassifier(base_score=0.5, booster='gbtree',
                                         colsample_bylevel=1,
                                         colsample_bynode=1, colsample_bytree=1,
                                         gamma=0, learning_rate=0.1,
                                         max_delta_step=0, max_depth=3,
                                         min_child_weight=1, missing=None,
                                         n_estimators=100, n_jobs=1,
                                         nthread=None,
                                         objective='binary:logistic',
                                         random_state=0, reg_alpha=0,
                                         reg_lambda=1, scale_pos_weight=1,
                                         seed=None, silent=None, subsample=1,
                                         verbosity=1),
                require_dense=[False, True])

In [ ]:
clf_CC.fit(X_train,y_train)

ClassifierChain(classifier=XGBClassifier(base_score=0.5, booster='gbtree',
                                         colsample_bylevel=1,
                                         colsample_bynode=1, colsample_bytree=1,
                                         gamma=0, learning_rate=0.1,
                                         max_delta_step=0, max_depth=3,
                                         min_child_weight=1, missing=None,
                                         n_estimators=100, n_jobs=1,
                                         nthread=None,
                                         objective='binary:logistic',
                                         random_state=0, reg_alpha=0,
                                         reg_lambda=1, scale_pos_weight=1,
                                         seed=None, silent=None, subsample=1,
                                         verbosity=1),
                order=None, require_dense=[False, True])

In [ ]:
clf_LP.fit(X_train,y_train)

## Model Evaluation

In [ ]:
#clf_CC = pickle.load(open('/content/drive/My Drive/BT4221/Models/CC_RandomForest.sav','rb'))
#clf_LP = pickle.load(open('/content/drive/My Drive/BT4221/Models/LP_RandomForest.sav','rb'))

In [ ]:
#prediction_BR = clf_BR.predict(X_test)
#prediction_CC = clf_CC.predict(X_test)
prediction_LP = clf_LP.predict(X_test)

In [ ]:
 #The fraction of labels that are incorrectly predicted
import sklearn.metrics as metrics

#print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_test, prediction_BR)))
#print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_test, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_test, prediction_LP)))

Hamming Loss (Binary Relevance) : 0.02221978412379967


In [ ]:
#print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_test, prediction_BR)))
#print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_test, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_test, prediction_LP)))

Accuracy Score (Binary Relevance) : 0.9129180903419956


In [ ]:
#print("F1 Score (Binary Relevance) : {}".format(f1_score(prediction_BR,y_test,average='micro')))
#print("F1 Score (Classification Chain) : {}".format(f1_score(prediction_CC,y_test,average='micro')))
print("F1 Score (Label Powerset) : {}".format(f1_score(prediction_LP,y_test,average='micro')))

F1 Score (Binary Relevance) : 0.5966808914177335


In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_XGBoost.sav','wb'))
#pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_XGBoost.sav','wb'))

In [ ]:
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_SVM.sav','wb'))